In [1]:
from astropy.io import fits
from reproject.mosaicking import find_optimal_celestial_wcs
from reproject import reproject_interp
from spectral_cube import SpectralCube

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
input_file = 'G301_HC3N_Tmb_DR1.fits'
output_file = 'test.fits'

cube = SpectralCube.read(input_file)
slice = cube[0:10]
slice.write(output_file, overwrite=True)

In [14]:
input_file = 'test.fits'
output_file = 'test_reprojected.fits'

hdu = fits.open(input_file)[0]

def check_if_fk5(hdu):
    if ('RA' in hdu.header['CTYPE1']) & ('DEC' in hdu.header['CTYPE2']):
        return True
    else:
        return False

def convert_fk5(hdu):

    print('Converting to FK5...')

    hdr = hdu.header
    hdu_2d = hdu.copy()
    hdu_2d.data = hdu_2d.data[0, :, :]
    hdu_2d.header['NAXIS'] = 2
    hdu_2d.header['WCSAXES'] = 2
    del hdu_2d.header['*3*']

    wcs_out, shape_out = find_optimal_celestial_wcs(hdu_2d, frame='fk5', auto_rotate=True, projection='GLS')
    # wcs_out, shape_out = find_optimal_celestial_wcs(hdu_2d, frame='fk5')

    hdr_out_ = wcs_out.to_header()

    hdr_out = hdr.copy()
    for key in ['CRPIX1', 'CRPIX2', 'PC1_1', 'PC1_2', 'PC2_1', 'PC2_2', 'CDELT1', 'CDELT2', 
                'CUNIT1', 'CUNIT2', 'CTYPE1', 'CTYPE2', 'CRVAL1', 'CRVAL2', 'LONPOLE', 'LATPOLE', 
                'RADESYS', 'EQUINOX']:
        try: 
            hdr_out[key] = hdr_out_[key]
        except: 
            print('Key not found:', key)
            continue
    del hdr_out['*PV*']

    array_out = reproject_interp(hdu, hdr_out, return_footprint=False)

    hdu_out = fits.PrimaryHDU(array_out, header=hdr_out)

    cube = SpectralCube.read(hdu_out)
    # cube.write(output_file, overwrite=True)

    return cube.hdu

if check_if_fk5(hdu):
    hdu = convert_fk5(hdu)

In [16]:
hdu.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                  775                                                  
NAXIS2  =                  398                                                  
NAXIS3  =                   10                                                  
DATAMIN = -0.1285860633850E+02         /                                        
DATAMAX =  0.1389085769653E+02         /                                        
BUNIT   = 'K       '           / Tmb                                            
OBJECT  = 'G301        '               /                                        
GLAT    =  0.3010000000000E+03         / Galactic latitude                      
GLON    =  0.0000000000000E+00         / Galactic longitude                     
EQUINOX =  0.2000000000000E+